<a href="https://colab.research.google.com/github/emmab-collab/cerebral-aneurysm-detection-3d/blob/main/notebooks/03_create_dictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

rsna_intracranial_aneurysm_detection_path = kagglehub.competition_download('rsna-intracranial-aneurysm-detection')
emmabenhaim_fonctions_u_path = kagglehub.dataset_download('emmabenhaim/fonctions-u')
emmabenhaim_cta_dataset_20_09_path = kagglehub.dataset_download('emmabenhaim/cta-dataset-20-09')

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd

import os
import ast
import glob
from scipy.ndimage import zoom
from tqdm import tqdm

#lire les images
import pydicom
import matplotlib.pyplot as plt

import random

In [ ]:
import sys
import numpy as np
sys.path.append("/kaggle/input/fonctions-u")  # adapte le nom exact

from utils import get_instance_number, coordonnee_z, get_center, get_pixelspacing, get_patient_ID, get_position
from utils import show_middle_slices,show_slice_with_point,ajouter_Modality
from utils import dicom_to_numpy,resample,crop,normalization,resample_coordonnees,preprocessing_volume_and_coords,preprocessing_volume
from utils import random_deformation, data_augmentation, dataset_augmented
from utils import extract_positives_cubes, extract_negative_cubes, get_non_overlapping_cubes, dictionnaire_patient, build_dataset_dict
from utils import train_model, combined_loss, evaluate_model, find_best_threshold

In [ ]:
df_loc = pd.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train_localizers.csv')
df_train = pd.read_csv('/kaggle/input/rsna-intracranial-aneurysm-detection/train.csv')
df_loc=ajouter_Modality(df_loc,df_train)
df_loc=df_loc[df_loc['Modality']=='CTA'].reset_index(drop=True)

# **Restructuration du dictionnaire CTA**

In [ ]:
loaded1= np.load("/kaggle/input/cta-dataset-20-09/CRA_dataset_20_09.npz", allow_pickle=True)
dict1 = {key: loaded1[key].item() for key in loaded1.files}

loaded2= np.load("/kaggle/input/cta-dataset-20-09/CTA2_dataset_20_09.npz", allow_pickle=True)
dict2 = {key: loaded2[key].item() for key in loaded2.files}

loaded3= np.load("/kaggle/input/cta-dataset-20-09/CTA3_dataset_20_09.npz", allow_pickle=True)
dict3 = {key: loaded3[key].item() for key in loaded3.files}

loaded4= np.load("/kaggle/input/cta-dataset-20-09/CTA4_dataset_20_09.npz", allow_pickle=True)
dict4 = {key: loaded4[key].item() for key in loaded4.files}

loaded5= np.load("/kaggle/input/cta-dataset-20-09/CTA5_dataset_20_09.npz", allow_pickle=True)
dict5 = {key: loaded5[key].item() for key in loaded5.files}

In [ ]:
loaded6= np.load("/kaggle/input/cta-dataset-20-09/CTA6_dataset_20_09.npz", allow_pickle=True)
dict6 = {key: loaded6[key].item() for key in loaded6.files}

loaded7 = np.load("/kaggle/input/cta-dataset-20-09/CTA7_dataset_20_09.npz", allow_pickle=True)
dict7 = {key: loaded7[key].item() for key in loaded7.files}

loaded8 = np.load("/kaggle/input/cta-dataset-20-09/CTA8_dataset_20_09.npz", allow_pickle=True)
dict8 = {key: loaded8[key].item() for key in loaded8.files}

loaded9 = np.load("/kaggle/input/cta-dataset-20-09/CTA9_dataset_20_09.npz", allow_pickle=True)
dict9 = {key: loaded9[key].item() for key in loaded9.files}

loaded10 = np.load("/kaggle/input/cta-dataset-20-09/CTA10_dataset_20_09.npz", allow_pickle=True)
dict10 = {key: loaded10[key].item() for key in loaded10.files}

In [ ]:
dicts = [dict1,dict2,dict3,dict4,dict5,dict6,dict7,dict8,dict9,dict10]
merged = {}
counter = 0

# Boucle sur chaque dictionnaire
for d in tqdm(dicts, desc="Dictionnaires"):
    # Boucle sur les paires clé/valeur du dictionnaire courant
    for _, value in tqdm(d.items(),
                         desc="Clés",
                         leave=False):  # leave=False pour éviter d'empiler les barres
        merged[f"patient_{counter}"] = value
        counter += 1

print(merged)

Clés:   0%|          | 0/197 [00:00<?, ?it/s]
                                             
Clés:   0%|          | 0/97 [00:00<?, ?it/s]
                                            
Clés:   0%|          | 0/98 [00:00<?, ?it/s]
                                            
Clés:   0%|          | 0/100 [00:00<?, ?it/s]
                                             
Clés:   0%|          | 0/98 [00:00<?, ?it/s]
                                            
Clés:   0%|          | 0/100 [00:00<?, ?it/s]
                                             
Clés:   0%|          | 0/98 [00:00<?, ?it/s]
                                            
Clés:   0%|          | 0/100 [00:00<?, ?it/s]
                                             
Clés:   0%|          | 0/99 [00:00<?, ?it/s]
                                            
Dictionnaires: 100%|██████████| 10/10 [00:00<00:00, 180.12it/s]


# **Travail avec un exemple**

In [ ]:
series_path = '/kaggle/input/rsna-intracranial-aneurysm-detection/series'

In [ ]:
series_path = '/kaggle/input/rsna-intracranial-aneurysm-detection/series'

# on parcourt une **copie** de l'index, sinon on a des problèmes en supprimant en direct
for i in tqdm(df_loc.index.tolist(), desc="Traitement DICOM"):
    patient_path = os.path.join(series_path, df_loc.loc[i, 'SeriesInstanceUID'])
    try:
        volume, aneurysm_coords = preprocessing_volume_and_coords(
            series_path, patient_path, df_loc
        )
    except AttributeError:
        # on supprime proprement la ligne qui a posé problème
        df_loc.drop(i, inplace=True)

# réinitialise l'index si tu veux un index continu
df_loc.reset_index(drop=True, inplace=True)

dictionnaire=build_dataset_dict(series_path, df_loc.iloc[:5], df_train)

for i in range(5):
    show_middle_slices(dictionnaire['patient_2']['cubes'][i])

In [ ]:
i=2
series_path='/kaggle/input/rsna-intracranial-aneurysm-detection/series'
patient_path = os.path.join(series_path, df_loc.iloc[i]['SeriesInstanceUID'])

volume,aneurysm_coords=preprocessing_volume_and_coords(series_path,patient_path,df_loc)
positive_cubes=extract_positives_cubes(volume, aneurysm_coords, N=5, cube_size=48, max_shift=20)

for cube in positive_cubes:
    show_middle_slices(cube)

# **Si le dictionnaire a l'air cohérent**

In [ ]:
dictionnaire=build_dataset_dict(series_path, df_loc.iloc[1000:], df_train,N_pos=3,N_neg=3)
np.savez("CTA10_dataset_20_09.npz", **dictionnaire)

 13%|█▎        | 28/223 [16:11<2:47:47, 51.63s/it]

Erreur pour 1.2.826.0.1.3680043.8.498.81805829917770672136502152837147205014: Seulement 0 cubes trouvés après 10000 essais.


 15%|█▍        | 33/223 [19:10<1:47:23, 33.91s/it]/usr/local/lib/python3.11/dist-packages/pydicom/pixels/utils.py:222: UserWarning: A value of 'None' for (0028,0008) 'Number of Frames' is invalid, assuming 1 frame
  warn_and_log(
 39%|███▉      | 87/223 [50:47<1:08:13, 30.10s/it]

Erreur pour 1.2.826.0.1.3680043.8.498.86319927242404542238601724247018344095: Seulement 0 cubes trouvés après 10000 essais.


 91%|█████████ | 202/223 [1:46:16<05:37, 16.06s/it]  

In [ ]:
loaded = np.load("/kaggle/input/cra1-dataset-20-09/CRA_dataset_20_09.npz", allow_pickle=True)
dictionnaire = {key: loaded[key].item() for key in loaded.files}

In [ ]:
len(dictionnaire['patient_0']['cubes'])

6

In [ ]:
series_path='/kaggle/input/rsna-intracranial-aneurysm-detection/series'
patient_path = os.path.join(series_path, dictionnaire['patient_2']['patient_ID'][0][0])
volume,aneurysm_coords=preprocessing_volume_and_coords(series_path,patient_path,df_loc)
show_slice_with_point(volume, aneurysm_coords, plane="axial")
positive_cubes=extract_positives_cubes(volume, aneurysm_coords, N=10, cube_size=48, max_shift=20)
for cube in positive_cubes:
    show_middle_slices(cube)

In [ ]:
for i in range(10):
    show_middle_slices(dictionnaire['patient_2']['cubes'][i])

In [ ]:
loaded = np.load("/kaggle/input/mra-dataset-18-09/MRA_dataset_16_09.npz", allow_pickle=True)
dictionnaire = {key: loaded[key].item() for key in loaded.files}

In [ ]:
series_path='/kaggle/input/rsna-intracranial-aneurysm-detection/series'
patient_path = os.path.join(series_path, dictionnaire['patient_271']['patient_ID'][0][0])
volume,aneurysm_coords=preprocessing_volume_and_coords(series_path,patient_path,df_loc)
show_slice_with_point(volume, aneurysm_coords, plane="axial")

positive_cubes=extract_positives_cubes(volume, aneurysm_coords, N=10, cube_size=48, max_shift=20)
for cube in positive_cubes:
    show_middle_slices(cube)

In [ ]:
for i in range(10):
    show_middle_slices(dictionnaire['patient_271']['cubes'][i])